# Quantifying N2O Emissions Reductions in Agricultural Crops through Nitrogen Fertilizer Rate Reduction (USA)

## VCS Methodology

### In this lab, the methodology by VCS (VM0022, Version 1.1) is used to calculate the number of verified carbon credits acquired from reduction of Nitrogen in the fertilizer. 

###### If you're using DATAIKU, run this lab with the following kernel: python env PEM 

Importing necessary modules

In [1]:
from __future__ import print_function
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, Layout
$ pip3 install voila

SyntaxError: invalid syntax (<ipython-input-1-67bf1d7e2584>, line 7)

Defining the fixed variables provided by the VCS paper based on the IPCC default 


In [0]:
#Emission factor for baseline direct N2O emissions from N inputs (imperical ton N2O–N (imperical ton N input)-1 ) Appendix F
EF_BDM1=0.01 

#Emission factor for project N2O emissions from N inputs (imperical ton N2O–N (imperical ton N input)-1 ) Appendix F)
EF_PDM1=0.01

#Global warming potential for N2O (imperical ton CO2e (imperical ton N2O)-1) Appendix F
N2O_GWP=310

#Fraction of all synthetic N added to baseline soils that volatilizes as NH3 and Nox (Dimensionless) Appendix F
FracGASF=0.10

#Fraction of all organic N added to baseline soils that volatilizes as NH3 and Nox (Dimensionless) Appendix F
FracGASM=0.20

#Fraction of N added (synthetic or organic) to baseline soils that is lost through leaching and runoff, in regions where leaching and runoff occurs (Dimensionless) Appendix A and F
FracLeach=0.30

#Emission factor for baseline N2O emissions from N leaching and runoff (imperical ton N2O–N (imperical ton N leached and runoff)^-1) Appendix F
EF_BIL=0.0075

#Emission factor for baseline N2O emissions from atmospheric deposition of N on soils and water surfaces  ([imperical ton N2O–N (imperical ton NH3–N + NOx–N volatilized)^-1]) Appendix F
EF_BIV=0.01

#Emission factor for project N2O emissions from atmospheric deposition of N on soils and water surfaces ([imperical ton N2O–N (imperical ton NH3–N + NOx–N volatilized)^-1]) Appendix F
EF_PIV=0.01

#Emission factor for project N2O emissions from N leaching and runoff (imperical ton N2O–N (imperical ton N leached and runoff)^-1) Appendix F
EF_PIL=0.0075

#Ratio molecular weight N2O/N
N2O_MW= 44.013/14.007 


The leakage deduction is assumed to be zero based on the methodology described in section 8.3. This can be changed if the Carbon Bank DSC team wishes to change this value in the future. 

In [0]:
#Leakage deduction- (set as 0 in this methodology, as described in section 8.3)
LK=0

The buffer deduction is assumed to be zero by the Carbon Bank DSC team. This is susceptible to change if the CarbonBank DSC team changes its mind. 

In [0]:
#Buffer deduction - (set as 0 in the methodology - this may not be an assumption by the CarbonBank)
BUF=0

# Sliders for input variables

The 'function' is now set up to calculate all the variables and graph the Total Emissions, Emission Reduction and Verified Carbon Credits.<br>
The variables of function 'function' require inputs for M_BSF, M_BOF, NC_BSF, NC_BOF, M_PSF,M_POF,NC_PSF,NC_POF, Area. <br> I want to create a graph with a slider so that the user can input their own values and playaround with different scenarios. <br> 
The graphs have already been created inside the function, but now I need to create the sliders. The cell below will create a slider for each input variable of the function. 

M=Mass
B=Baseline
P=Project
S=Synthetic
O=Organic
F=Fertilizer

### Data range
Before we make the sliders, we need to have a notion of the range of the input variables. <br>

##### Mass range
To gain an idea of the range of value of the Mass and N content:
https://www.ers.usda.gov/data-products/fertilizer-use-and-price.aspx (table 10) shows that the average amount of Nitrogen used on corn in the latests year (2018) is 149pounds/acre. <br> 
Since kg ha-1 = kilograms per hectare = 0.893 pounds per acre <br>
149pounds/acre=0.167Mg/ha <br>
The range chosen for the slider is therefore chosen to be: min_mass=0 max_mass=0.3

##### N content  range
N content (Nitrogen content in grams per 100g of fertilizer used) basically is the percentage of Nitrogen of the fertilizer used. Using logic, the minimum amount of Nitrogen a fertizilier could have is 0% and maximum is 100%. <br>
The range chosen for the slider is therefore: min_NC=0 max_NC=1

In [0]:
#making sure the entire text in the description of the slider shows
style = {'description_width': 'initial'}

#defining the minimum and maximum values for the sliders
min_mass=0
max_mass=0.9
min_NC=0
max_NC=1
min_area=1
max_area=200

##CREATING THE SLIDERS FOR THE GRAPHS
#Slider for the mass of baseline synthetic fertilizer
M_BSF_Slider=FloatSlider(
    min=min_mass,
    max=max_mass,
    step=0.001,
    description=' Mass of baseline N containing synthetic fertilizer applied (imperical ton ha-1 in year t)….',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_mass-min_mass)/2,
    layout=Layout(width='800px'))

#Slider for the mass of baseline organic fertilizer
M_BOF_Slider=FloatSlider(
    min=min_mass,
    max=max_mass,
    step=0.001,
    description=' Mass of baseline N containing organic fertilizer applied (imperical ton ha-1 in year t)…..',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_mass-min_mass)/2,
    layout=Layout(width='800px'))

#Slider for the N content of baseline synthetic fertilizer
NC_BSF_Slider=FloatSlider(
    min=min_NC,
    max=max_NC,
    step=0.01,
    description='Percentage of Nitrogen in synthetic fertilizer applied during baseline (fraction)..............',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_NC-min_NC)/2,
    layout=Layout(width='800px'))

#Slider for the N content of baseline organic fertilizer
NC_BOF_Slider=FloatSlider(
    min=min_NC,
    max=max_NC,
    step=0.01,
    description='Percentage of Nitrogen in organic fertilizer applied during baseline (fraction)................',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_NC-min_NC)/2,
    layout=Layout(width='800px'))

#Slider for the mass of project synthetic fertilizer
M_PSF_Slider=FloatSlider(
    min=min_mass,
    max=max_mass,
    step=0.001,
    description='Mass of project N containing synthetic fertilizer applied (imperical ton ha-1 in year t)….',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_mass-min_mass)/2,
    layout=Layout(width='800px'))

#Slider for the mass of project organic fertilizer
M_POF_Slider=FloatSlider(
    min=min_mass,
    max=max_mass,
    step=0.001,
    description='Mass of project N containing organic fertilizer applied (imperical ton ha-1 in year t)……',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_mass-min_mass)/2,
    layout=Layout(width='800px'))

#Slider for the N content of project synthetic fertilizer
NC_PSF_Slider=FloatSlider(
    min=min_NC,
    max=max_NC,
    step=0.01,
    description='Percentage of Nitrogen in synthetic fertilizer applied during project (fraction)..............',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_NC-min_NC)/2,
    layout=Layout(width='800px'))

#Slider for the N content of project organic fertilizer
NC_POF_Slider=FloatSlider(
    min=min_NC,
    max=max_NC,
    step=0.01,
    description='Percentage of Nitrogen in organic fertilizer applied during project (fraction)................',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_NC-min_NC)/2,
    layout=Layout(width='800px'))

#Slider for the area of the proejct
Area_Slider=FloatSlider(
    min=min_area,
    max=max_area,
    step=1,
    description='Project Area (ha).....................................................................................................',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style,
    value=(max_area-(min_area-1))/2,
    layout=Layout(width='800px'))


# Baseline Emissions

In [0]:
def BaselineCalculation(M_BSF, M_BOF, NC_BSF, NC_BOF, Area):
    
    #SYNTHETIC AND ORGANIC FERTILIZER APPLIED IN BASELINE
    #Synthetic fertilizer applied in baseline (the same for the two methods)
    F_BSN = M_BSF * NC_BSF 
    #Organic fertilizer applied in baseline (the same for the two methods)
    F_BON = M_BOF * NC_BOF
    
    ##DIRECT BASELINE EMISSIONS FOR METHOD 1 AND 2
    ##DIRECT BASELINE EMISSIONS METHOD 1
    #Direct baseline nitrous oxide emissions from nitrogen fertilization for method 1
    N2O_B_Dir1=(F_BON+F_BSN)* EF_BDM1 * N2O_MW * N2O_GWP
    
    ##DIRECT BASELINE EMISSIONS METHOD 2
    #Emission factor of baseline direct emission, method 2
    EF_BDM2=0.00067*((math.exp(6.7*(F_BSN+F_BON))-1)/(F_BSN+F_BON))
    #Direct emissions of nitrous oxide from method 2
    N2O_B_Dir2=(F_BON+F_BSN)* EF_BDM2 * N2O_MW * N2O_GWP
    
    
    #INDIRECT BASELINE EMISSIONS (both methods)
    #Nitrous oxide indirect emissions volatization (the same for the two methods)
    N2O_B_Vol=((F_BSN * FracGASF ) + (F_BON * FracGASM)) * EF_BIV  * N2O_MW * N2O_GWP
    #Nitrous oxide indirect emossions from leaching and runoff (the same for the two methods)
    N2O_B_Leach= (F_BSN+F_BON) * FracLeach * EF_BIL* N2O_MW * N2O_GWP
    #Nitrous oxide baseline indirect emissions (the same for the two methods)
    N2O_B_Ind= N2O_B_Vol + N2O_B_Leach
    
    
    #TOTAL BASELINE EMISSIONS METHOD 1 AND 2
    #Total Emissions Baseline with method 1
    N2O_B_Total1= N2O_B_Dir1 + N2O_B_Ind
    #Total Emissions Baseline with method 2
    N2O_B_Total2= N2O_B_Dir2 + N2O_B_Ind
    
        
    #VISUALIZATION CHART OF TOTAL EMISSIONS PER AREA (Ha)
    #preparing vectors for bar chart of total emissions
    labelsE=['Method 1', 'Method 2']
    EBHa=[N2O_B_Total1,N2O_B_Total2] #emission baseline per Ha vector (imperical ton CO2e ha-1 in year t) to (imperical ton CO2e in year t)
    #creating, formatting and labelling bar chart of total emissions
    x = np.arange(len(labelsE))
    width = 0.25
    Ebar=plt.figure(1) #total emissions bar graph
    plt.bar(labelsE,EBHa)
    plt.xticks(x,labelsE)
    #plt.legend(labelsE,fontsize=12)
    plt.ylabel('Total Emissions per Ha (imperical ton CO2e ha-1 in year t)',fontsize=12)
    plt.title('Nitrous Oxide Total Emissions per Ha',fontsize=15)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    Ebar.show()
    
    
    #VISUALIZATION CHART OF TOTAL EMISSIONS FOR TOTAL PROJECT AREA
    #preparing vectors for bar chart of total emissions
    labelsE=['Method 1', 'Method 2']
    EBTA=[N2O_B_Total1*Area,N2O_B_Total2*Area] #vector emission for total area baseline. The vector multiplied by area to transform units from (imperical ton CO2e ha-1 in year t) to (imperical ton CO2e in year t)
    #creating, formatting and labelling bar chart of total emissions
    x = np.arange(len(labelsE))
    width = 0.25
    Ebar=plt.figure(2) #total emissions bar graph
    plt.bar(x,EBTA)
    plt.xticks(x,labelsE)
    #plt.legend(labelsE,fontsize=12)
    plt.ylabel('Total Emissions (imperical ton CO2e in year t)',fontsize=12)
    plt.title('Nitrous Oxide Total Emission for Project Area',fontsize=15)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    Ebar.show()   
    
    

In [0]:
#Slider to update bar graph including range of each variable    
interact(BaselineCalculation,M_BSF=M_BSF_Slider, M_BOF=M_BOF_Slider, NC_BSF=NC_BSF_Slider, NC_BOF=NC_BOF_Slider, Area=Area_Slider);

# Baseline emissions, Project Emissions, Emissions Reductions and VCUs

The following blox of text will contain a user defined function which includes the following:
- Defining the equations from the VSC Methodology PDF
- Plotting the number total emissions from baseline, project, and reduction for both methods
- Plotting the number percentage change in emission reduction between the baseline and project for both methods
- Plotting the number of Verified Carbon Credits for both methods

Note:
The only mathematical difference between the two methods is the Emission Factor. The emission factor for method 1 is fixed and defined above (EF_BDM1,EF_PDM1,EF_BIV,EF_PIV,EF_PIL). The emission factor for method 2 is calculated below for the baseline (EF_BDM2) and proejct (EF_PDM2)

In [0]:
def function(M_BSF, M_BOF, NC_BSF, NC_BOF, M_PSF,M_POF,NC_PSF,NC_POF, Area):
    
    #SYNTHETIC AND ORGANIC FERTILIZER APPLIED IN BASELINE
    #Synthetic fertilizer applied in baseline (the same for the two methods)
    F_BSN = M_BSF * NC_BSF 
    #Organic fertilizer applied in baseline (the same for the two methods)
    F_BON = M_BOF * NC_BOF
    
    
    ##DIRECT BASELINE EMISSIONS FOR METHOD 1 AND 2
    ##DIRECT BASELINE EMISSIONS METHOD 1
    #Direct baseline nitrous oxide emissions from nitrogen fertilization for method 1
    N2O_B_Dir1=(F_BON+F_BSN)* EF_BDM1 * N2O_MW * N2O_GWP
    
    ##DIRECT BASELINE EMISSIONS METHOD 2
    #Emission factor of baseline direct emission, method 2
    EF_BDM2=0.00067*((math.exp(6.7*(F_BSN+F_BON))-1)/(F_BSN+F_BON))
    #Direct emissions of nitrous oxide from method 2
    N2O_B_Dir2=(F_BON+F_BSN)* EF_BDM2 * N2O_MW * N2O_GWP
    
    
    #INDIRECT BASELINE EMISSIONS (both methods)
    #Nitrous oxide indirect emissions volatization (the same for the two methods)
    N2O_B_Vol=((F_BSN * FracGASF ) + (F_BON * FracGASM)) * EF_BIV  * N2O_MW * N2O_GWP
    #Nitrous oxide indirect emossions from leaching and runoff (the same for the two methods)
    N2O_B_Leach= (F_BSN+F_BON) * FracLeach * EF_BIL* N2O_MW * N2O_GWP
    #Nitrous oxide baseline indirect emissions (the same for the two methods)
    N2O_B_Ind= N2O_B_Vol + N2O_B_Leach
    
    
    #TOTAL BASELINE EMISSIONS METHOD 1 AND 2
    #Total Emissions Baseline with method 1
    N2O_B_Total1= N2O_B_Dir1 + N2O_B_Ind
    #Total Emissions Baseline with method 2
    N2O_B_Total2= N2O_B_Dir2 + N2O_B_Ind
    
    
    #SYNTHETIC AND ORGANIC FERTILIZER APPLIED IN PROJECT
    #synthetic fertilizer applied in project
    F_PSN = M_PSF * NC_PSF
    #organic fertilizer applied in project 
    F_PON = M_POF * NC_POF

    
    #DIRECT PROJECT EMISSIONS FOR METHOD 1 AND 2
    #DIRECT PROJECT EMISSIONS METHOD 1
    #direct project nitrous oxide direct emissions with method 1
    N2O_P_Dir1=(F_PON+F_PSN)* EF_PDM1 * N2O_MW * N2O_GWP
    #DIRECT PROJECT EMISSIONS METHOD 2
    #emission factor for method 2 needed to calculate direct nitrous oxide direct emissions with method 2
    EF_PDM2=0.00067*((math.exp(6.7*(F_PSN+F_PON))-1)/(F_PSN+F_PON))
    #direct project nitrous oxide direct emissions with method 2
    N2O_P_Dir2=(F_PON+F_PSN)* EF_PDM2 * N2O_MW * N2O_GWP
    
    
    #INDIRECT PROJECT EMISSIONS (BOTH METHODS)
    #Nitrous oxide indirect emissions volatization (the same for the two methods)
    N2O_P_Vol=((F_PSN * FracGASF ) + (F_PON * FracGASM)) * EF_PIV * N2O_MW * N2O_GWP
    #Nitrous oxide indirect emossions from leaching and runoff (the same for the two methods)
    N2O_P_Leach= (F_PSN+F_PON) * FracLeach * EF_PIL * N2O_MW * N2O_GWP
    #Nitrous oxide baseline indirect emissions (the same for the two methods)
    N2O_P_Ind= N2O_P_Vol + N2O_P_Leach
    
    
    #TOTAL PROJECT EMISSIONS METHOD 1 AND 2
    #Total Emissions Project with method 1
    N2O_P_Total1= N2O_P_Dir1 + N2O_P_Ind
    #Total Emissions Project with method 2
    N2O_P_Total2= N2O_P_Dir2 + N2O_P_Ind
    

    #UNCERTAINTY DEDUCTION 
    #Calculating the uncertainty deduction from section 8.4 of the VSC PDF
    UncRange=(1-(0.63*math.exp(-40*((F_PSN+F_PON)**2))))*100 #equation for uncertainity in N2O emission reductions
    if UncRange<=15: 
        Unc=0 #uncertainty deduction for uncertaintiy range below 15% based on table 3 of section 8.4
    elif 15<=UncRange<=30:
        Unc=0.057 #uncertainty deduction for uncertaintiy range above 15% and above 30% based on table 3 of section 8.4
    elif 30<=UncRange>=50:
        Unc=0.107 #uncertainty deduction for uncertaintiy range above 30% and above 50% based on table 3 of section 8.4
    else:
        Unc=0.164 #uncertainty deduction for uncertaintiy range above 50% based on table 3 of section 8.4
    
    
    #REDUCTION IN TOTAL EMISSIONS METHOD 1 AND 2
    #Reduction in total emission method 1
    N2O_PR1= (N2O_B_Total1 - N2O_P_Total1) * Area * (1-LK) * (1-Unc)
    #Reduction in total emission method 2
    N2O_PR2=(N2O_B_Total2 - N2O_P_Total2) * Area * (1-LK) * (1-Unc)

    
    #NUMBER OF VERIFIED CARBON CREDITS WITH METHOD 1 AND 2
    #Number of verified carbon credits with method 1
    VCC1= N2O_PR1 * (1-BUF)
    #Number of verified carbon credits with method 2
    VCC2= N2O_PR2 * (1-BUF)
    
    #
    #
    #
    #
    #ALL THE CALCULATIONS FROM VCS ARE NOW COMPLETE
    #FOLLOWING CODE IS TO VISUALIZE DATA
    
    
        
    #VISUALIZATION CHART OF TOTAL EMISSIONS PER AREA (Ha)
    #preparing vectors for bar chart of total emissions
    labelsE=['Method 1', 'Method 2']
    EB=[N2O_B_Total1,N2O_B_Total2] #emission baseline vector  (imperical ton CO2e ha-1 in year t) to (imperical ton CO2e in year t)
    EP=[N2O_P_Total1,N2O_P_Total2] #emission project vector (imperical ton CO2e ha-1 in year t) 
    ER=[N2O_PR1/Area, N2O_PR2/Area] #emission reduction vector divided by area to transform unit from (imperical ton CO2e in year t) to (imperical ton CO2e ha-1 in year t)
    #creating, formatting and labelling bar chart of total emissions
    x = np.arange(len(labelsE))
    width = 0.25
    Ebar=plt.figure(1) #total emissions bar graph
    plt.bar(x-width,EB,width)
    plt.bar(x,EP,width)
    plt.bar(x+width,ER,width)
    plt.xticks(x,labelsE)
    plt.legend(['Baseline','Project','Emission Reduction'],fontsize=12)
    plt.ylabel('Total Emissions per Ha (imperical ton CO2e ha-1 in year t)',fontsize=12)
    plt.title('Nitrous Oxide Total Emissions per Ha',fontsize=15)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    Ebar.show()
    
    #VISUALIZATION CHART OF TOTAL EMISSIONS FOR TOTAL PROJECT AREA
    #preparing vectors for bar chart of total emissions
    labelsE=['Method 1', 'Method 2']
    EB=[N2O_B_Total1*Area,N2O_B_Total2*Area] #emission baseline vector multiplied by area to transform units from (imperical ton CO2e ha-1 in year t) to (imperical ton CO2e in year t)
    EP=[N2O_P_Total1*Area,N2O_P_Total2*Area] #emission project vector multiplied by area to transform units from (imperical ton CO2e ha-1 in year t) to (imperical ton CO2e in year t)
    ER=[N2O_PR1, N2O_PR2] #emission reduction vector
    #creating, formatting and labelling bar chart of total emissions
    x = np.arange(len(labelsE))
    width = 0.25
    Ebar=plt.figure(2) #total emissions bar graph
    plt.bar(x-width,EB,width)
    plt.bar(x,EP,width)
    plt.bar(x+width,ER,width)
    plt.xticks(x,labelsE)
    plt.legend(['Baseline','Project','Emission Reduction'],fontsize=12)
    plt.ylabel('Total Emissions (imperical ton CO2e in year t)',fontsize=12)
    plt.title('Nitrous Oxide Total Emission for Project Area',fontsize=15)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    Ebar.show()
    
    
    
    #VISUALIZATION OF EMISSION REDUCTION PERCENTAGE
    #calculating Emission Reduction Percentage (ERP) percentage change for method 1
    ERP1=((N2O_P_Total1-N2O_B_Total1)/N2O_P_Total1)*-100 #what are the units?
    #calculating percentage change for method 2
    ERP2=((N2O_P_Total2-N2O_B_Total2)/N2O_P_Total2)*-100
    #creating, formatting and labelling bar chart of emission reductions
    ERP=[ERP1,ERP2]
    ERPbar=plt.figure(3)
    plt.bar(labelsE,ERP,color='green')
    plt.title("Emission Reduction (%)",fontsize=15)
    plt.xlabel('Method',fontsize=12)
    plt.ylabel('Emission Reduction (%)',fontsize=12)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    ERPbar.show()
    
    #VISUALIZATION CHART OF VERIFIED CARBON CREDITS
    #preparing vectors for bar chart of carbon credits
    labelsE=['Method 1', 'Method 2']
    VCC=[VCC1, VCC2] #VCC as a vector
    #creating, formatting and labelling bar chart of carbon credits
    VCCbar=plt.figure(4)
    plt.bar(labelsE,VCC,color='red')
    plt.title("Verified Carbon Units at time t (imperical tonCo2e) for project area",fontsize=15)
    plt.xlabel('Method',fontsize=12)
    plt.ylabel('Verified Carbon Units',fontsize=12)
    plt.figure(2)
    plt.grid(True, color = "grey", linewidth = "0.3", linestyle = "-")
    VCCbar.show()

The 'function' is now set up to calculate all the variables and graph the Total Emissions, Emission Reduction and Verified Carbon Credits. <br>The sliders for each variable is now defined. <br> We are now going to use ipywidgets module 'interact' to call the function. The input variables of the function are going to call the sliders created in the cell above.

In [0]:
#Slider to update bar graph including range of each variable              
interact(function,M_BSF=M_BSF_Slider, M_BOF=M_BOF_Slider, NC_BSF=NC_BSF_Slider, NC_BOF=NC_BOF_Slider, M_PSF=M_PSF_Slider,M_POF=M_POF_Slider,NC_PSF=NC_PSF_Slider,NC_POF=NC_POF_Slider, Area=Area_Slider);